In [1]:
from utils import *
from vgg16 import Vgg16

Using Theano backend.


In [2]:
path = "../data/dogscats/sample"

In [3]:
vgg = Vgg16()
vgg.compile()

/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/taot/opt/miniconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [4]:
batches = vgg.get_batches(path + '/train', batch_size=16)
val_batches = vgg.get_batches(path + '/valid', batch_size = 16)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [5]:
# vgg.model.summary()

In [9]:
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=4)

Epoch 1/1
16/16 [==============================] - 6s - loss: 1.1830 - acc: 0.6250 - val_loss: 0.5302 - val_acc: 0.8750


In [14]:
vgg.model.save_weights(path + '/models/dogscats/dogscats_vgg16.h5')

In [ ]:
vgg.model.summary()

In [31]:
layers = vgg.model.layers
last_conv_index = [index for index, layer in enumerate(layers) if type(layer) is Convolution2D][-1]

In [32]:
layers[last_conv_index]

In [58]:
conv_layers = layers[:last_conv_index+1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_index+1:]

In [36]:
batches = vgg.get_batches(path + '/train', batch_size=16)
val_batches = vgg.get_batches(path + '/valid', batch_size = 16)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [40]:
trn_classes = batches.classes
val_classes = val_batches.classes
trn_labels = onehot(trn_classes)
val_labels = onehot(val_classes)

In [42]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [43]:
trn_features= conv_model.predict_generator(batches, batches.nb_sample)

In [47]:
save_array(path + '/models/dogscats/conv_val_features.bc', val_features)
save_array(path + '/models/dogscats/conv_trn_features.bc', trn_features)

In [76]:
# why do we need to half the weights? Need to test.
def proc_weights(layer):
    return [o / 2 for o in layer.get_weights()]

In [56]:
opt = RMSprop(lr=0.00001, rho=0.7)

In [74]:
def get_fc_model():
    last_output_shape = conv_layers[-1].output_shape[1:]
    # Can we remove the two dropout layers?
    model = Sequential([
        MaxPooling2D(input_shape=last_output_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.0),
        Dense(4096, activation='relu'),
        Dropout(0.0),
        Dense(2, activation='softmax')
    ])
    for l1, l2 in zip(model.layers, fc_layers):
        l1.set_weights(proc_weights(l2))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [78]:
fc_model = get_fc_model()
fc_model

In [80]:
fc_model.fit(trn_features, trn_labels, nb_epoch=3, batch_size=16, validation_data=(val_features, val_labels))

Train on 16 samples, validate on 8 samples
Epoch 1/3
16/16 [==============================] - 0s - loss: 0.0725 - acc: 1.0000 - val_loss: 0.6084 - val_acc: 0.6250
Epoch 2/3
16/16 [==============================] - 0s - loss: 0.0471 - acc: 1.0000 - val_loss: 0.6037 - val_acc: 0.6250
Epoch 3/3
16/16 [==============================] - 0s - loss: 0.0321 - acc: 1.0000 - val_loss: 0.6016 - val_acc: 0.7500
